# Imports

In [1]:
import re
import string

import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 150)

from preprocessing import tokenize

In [2]:
DATA_PATH_PREP = '../DATA/prepared'

# Load data

In [3]:
df_full = pd.read_pickle(f'{DATA_PATH_PREP}/02_df_full_text_no_meta.pkl')
df_full.head()

,author,title,download_link,local_filename,text
0,ivan_vazov,Чичовци,https://chitanka.info/text/3757-chichovtsi.txt.zip,../DATA/original/ivan_vazov_tbc_chichovtsi.txt,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
1,ivan_vazov,Под игото,https://chitanka.info/text/3753-pod-igoto.txt.zip,../DATA/original/ivan_vazov_tbc_pod_igoto.txt,"\n\n\n\tЧаст първа\n\n\tI. Гост\n\n\tТая прохладна майска вечер чорбаджи Марко, гологлав, по халат, вечеряше с челядта си на двора.\n\tГосподарска..."
2,ivan_vazov,Българският език,https://chitanka.info/text/5189-bylgarskijat-ezik.txt.zip,../DATA/original/ivan_vazov_bylgarskijat_ezik.txt,"\n\n\n\n\tЕзик свещен на моите деди\n\tезик на мъки, стонове вековни,\n\tезик на тая, дето ни роди\n\tза радост не — за ядове отровни.\n\n\tЕзик п..."
3,ivan_vazov,Кочо,https://chitanka.info/text/3851-kocho.txt.zip,../DATA/original/ivan_vazov_kocho.txt,"\n\tЗащитата на Перущица\n\n\n\tO, движенье славно, о, мрачно движенье,\n\tдни на борба горда, о, дни на паденье!\n\tЕпопея тъмна, непозната нам,\..."
4,ivan_vazov,Левски,https://chitanka.info/text/3849-levski.txt.zip,../DATA/original/ivan_vazov_levski.txt,"\n\n\n\tМанастирът тесен за мойта душа е.\n\tКога човек дойде тук да се покае,\n\tтрябва да забрави греховния мир,\n\tда бяга съблазни и да търси ..."


# Create samples

In [4]:
df_group_by_author = df_full.groupby("author")["text"].agg(" ".join).reset_index()
df_group_by_author

,author,text
0,aleko-konstantinov,"\n\n\n\tНашият помощник на регистратора е страшен комик. Изпокапваме от смях, когато почне да бомбардира несправедливата съдба.\n\t— Е, дявол да г..."
1,dimityr-dimov,"\n\n\n\tI глава\n\n\tГроздоберът беше към края си.\n\tОт малките вили и кирпичените постройки сред лозята се разнасяха ту задружни песни, бързи и ..."
2,dimityr-talev,"\n\n\n\tПърва част\n\tХаджи Серафимовата внука\n\n\n\n\tОвде дърво столовито,\n\tстоловито, грановито,\n\tгранки му са до небеси,\n\tа корени — су..."
3,elin-pelin,"\n\n\n\tПърва глава\n\n\tНай-заможният човек в селото беше дядо Йордан Геракът. Пъргав и трудолюбив, той бе работил през целия си живот и бе сполу..."
4,ivan_vazov,\n\tГалерия от типове и нрави български в турско време\n\n\n\tI. Общество\n\n\tУтринното лятно слънце изскокна високо над Стара планина. Потоци от...
5,jordan-jovkov,"\n\n\n\tI\n\n\tГрадът бързо се подновяваше и растеше. Но все пак оставаха много улици, където още можеше да се видят тихи, едновремешни къщурки, с..."


In [5]:
limit_x = df_group_by_author['text'].map(lambda t: len(t.strip().split())).min() - 1000
print(f'{limit_x=}')
limit_y = 150_000

num_authors = df_group_by_author.shape[0]
num_samples = 30

start_idxs = np.random.randint(0, limit_x, (num_authors, num_samples))
sample_lengths = np.random.randint(0, limit_y, (num_authors, num_samples))

df_samples = pd.DataFrame(columns=['text', 'author'])
for i in range(num_authors):
    author_name = df_group_by_author['author'][i]
    for j, start_idx in enumerate(start_idxs[i]):
        sample = df_group_by_author['text'][i][start_idx : (start_idx + sample_lengths[i, j])]
        df_samples.loc[len(df_samples)] = [sample, author_name]
df_samples

limit_x=21623


,text,author
0,"ли“. Публикуван е за първи път в „Знаме“, г. II, бр.79 от 5 февруари 1897 г., с подпис_ Щастливец.\n\t_В началото на 1897 г. агентите на сръбската...",aleko-konstantinov
1,"ози Биконсфилд, дето ни разпокъса. Дип добра си беше Санстефанска България, защо му трябваше да я цепи. Иди сега, че мигай на празни ясли, иди, че...",aleko-konstantinov
2,"тук ще се бавим цял час, аз преспокойно се разположих при една маса и си поръчах закуска и пиво. Наоколо ми свят — гъмжило! Па и хубав свят. Мадж...",aleko-konstantinov
3,"сеш…\n\t— За какъв инат приказваш бе, чичо?\n\t— Как за какъв инат бе, синко. Ами че не се ли сещаш, не се ли виждаш на кой хал си стигнал. На теб...",aleko-konstantinov
4,"еш и ти наред с хората. Я погледай твоите връстници: всички на работа са наредени, и парици си спастриха, и къщици си издигнаха, а пък ти!… Я се в...",aleko-konstantinov
...,...,...
175,", които се пристягаха в новите си палта и закопчаваха всичките си копчета. Празникът личеше във всичко. И слънцето по-ярко светеше, и небето по-ху...",jordan-jovkov
176,"той беше безумно влюбен, и то в най-хубавото момиче в града — Цветана. Тя беше дъщеря на един от най-първите чифликчии в околността. По една откол...",jordan-jovkov
177,"следи модата през всички епохи. Но това не смущаваше никоя от тях и тъй като всяко домашно огледало, като стар и послушен слуга, успяваше да скрие...",jordan-jovkov
178,"якой недостатък в тоалета им — това никой не можеше да каже, защото младостта и разцъфналата хубост изкупваха всичко друго.\n\tНо първенството в б...",jordan-jovkov


# Tokenize

In [6]:
test = 'Това е низ! Съдържа препинателни "знаци" и цифри като: 42, 420, 4200 и 7.\nКак можем да го разделим??'

print('################ Soft ################')
print(tokenize(test, 'soft'))
print()

print('################ Medium ################')
print(tokenize(test, 'medium'))
print()

print('################ Hard ################')
print(tokenize(test, 'hard'))

################ Soft ################
['това', 'е', 'низ', '!', 'съдържа', 'препинателни', '"', 'знаци', '"', 'и', 'цифри', 'като', ':', '42', ',', '420', ',', '4200', 'и', '7', '.', 'как', 'можем', 'да', 'го', 'разделим', '?', '?', 'това е', 'е низ', 'низ !', '! съдържа', 'съдържа препинателни', 'препинателни "', '" знаци', 'знаци "', '" и', 'и цифри', 'цифри като', 'като :', ': 42', '42 ,', ', 420', '420 ,', ', 4200', '4200 и', 'и 7', '7 .', '. как', 'как можем', 'можем да', 'да го', 'го разделим', 'разделим ?', '? ?', 'това е низ', 'е низ !', 'низ ! съдържа', '! съдържа препинателни', 'съдържа препинателни "', 'препинателни " знаци', '" знаци "', 'знаци " и', '" и цифри', 'и цифри като', 'цифри като :', 'като : 42', ': 42 ,', '42 , 420', ', 420 ,', '420 , 4200', ', 4200 и', '4200 и 7', 'и 7 .', '7 . как', '. как можем', 'как можем да', 'можем да го', 'да го разделим', 'го разделим ?', 'разделим ? ?', 'това е низ !', 'е низ ! съдържа', 'низ ! съдържа препинателни', '! съдържа препин

In [7]:
df_full_soft = df_samples.copy()
df_full_soft['text'] = df_full_soft['text'].map(lambda t: tokenize(t, 'soft'))

df_full_med = df_samples.copy()
df_full_med['text'] = df_full_med['text'].map(lambda t: tokenize(t, 'medium'))

df_full_hard = df_samples.copy()
df_full_hard['text'] = df_full_hard['text'].map(lambda t: tokenize(t, 'hard'))

In [8]:
print('################ Heads ################')
print(df_full_soft['text'][0][:20])
print(df_full_med['text'][0][:20])
print(df_full_hard['text'][0][:20])

print('################ Tails ################')
print(df_full_soft['text'][0][-20:])
print(df_full_med['text'][0][-20:])
print(df_full_hard['text'][0][-20:])

################ Heads ################
['ли', '“', '.', 'публикуван', 'е', 'за', 'първи', 'път', 'в', '„', 'знаме', '“', ',', 'г', '.', 'ii', ',', 'бр', '.', '79']
['ли', '“', '.', 'публикувам', 'е', 'за', 'първи', 'път', 'в', '„', 'знаме', '“', ',', 'г', '.', 'ii', ',', 'бр', '.', '79']
['публикува', 'път', 'февруар', 'подпис', 'щастлив', 'нача', 'агент', 'сръбск', 'пропаганд', 'македони', 'пове', 'ожесточ', 'борб', 'назначаван', 'българи', 'синесий', 'митрополит', 'скопи', 'тогаваш', 'българск']
################ Tails ################
['хич олмаса , тоя', 'олмаса , тоя дангалак', ', тоя дангалак е', 'тоя дангалак е изял', 'дангалак е изял някои', 'е изял някои …', 'изял някои … някоя', 'някои … някоя попара', '… някоя попара .', 'някоя попара . “', 'попара . “ иностранците', '. “ иностранците тълкуваха', '“ иностранците тълкуваха по', 'иностранците тълкуваха по своему', 'тълкуваха по своему тези', 'по своему тези мимики', 'своему тези мимики и', 'тези мимики и едва', 'мимики и едва 

In [9]:
df_full_hard.head()

,text,author
0,"[публикува, път, февруар, подпис, щастлив, нача, агент, сръбск, пропаганд, македони, пове, ожесточ, борб, назначаван, българи, синесий, митрополит...",aleko-konstantinov
1,"[ог, биконсфилд, дето, разпокъса, дип, санстефанск, българ, трябва, цепя, ида, мига, празн, ясл, ида, чака, усет, освобод, па, рус, хас, джанъм, р...",aleko-konstantinov
2,"[бавя, цял, час, преспоко, разполож, маса, поръча, закуск, пиво, наокол, свят, гъмж, па, хубав, свят, маджар, зная, твърд, сърц, ама, видя, маджар...",aleko-konstantinov
3,"[ина, приказва, чичо, ина, синк, ами, сеща, вижда, хал, стигн, теб, оста, свет, оправя, бре, чедо, свест, па, издуха, вятър, глава, па, тръгн, нар...",aleko-konstantinov
4,"[а, наред, хора, погледа, твои, връстниц, работ, наред, париц, спастр, къщиц, издигн, видя, заприлича, трън, потътр, заден, малъ, мисл, вижда, обр...",aleko-konstantinov


# Saving to files

In [10]:
df_full_soft.to_pickle(f'{DATA_PATH_PREP}/03_df_full_soft.pkl')
df_full_med.to_pickle(f'{DATA_PATH_PREP}/03_df_full_med.pkl')
df_full_hard.to_pickle(f'{DATA_PATH_PREP}/03_df_full_hard.pkl')